In [ ]:
This notebook was created using https://bigsql.com/[BigSQL] and http://conda.pydata.org/miniconda.html[Miniconda]. If you want to follow those steps, install both of those packages first. After Miniconda is installed, run the following commands:

````
cd <location of miniconda install>
cd bin
./conda create --name PGDay
source activate PGDay
````

You should now have an active conda environment called PGDay, and your prompt should indicate this:

````
(PGDay) decibel@decina:[17:24]~/git/PGDay2016
````

Now we can install jupyter and run it:

````
conda install jupyter
cd <location of PGDay2016 git repository>
jupyter notebook Python+Postgres.ipynb
````

One last thing to do: install pip and jupyter_sql

In [1]:
!conda install -y sqlalchemy pip

Fetching package metadata .......
Solving package specifications: ..........

Package plan for installation in environment /Users/decibel/miniconda3/envs/PGDay:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    sqlalchemy-1.1.3           |           py35_0         1.5 MB

The following NEW packages will be INSTALLED:

    sqlalchemy: 1.1.3-py35_0

Fetching packages ...
sqlalchemy-1.1 100% |################################| Time: 0:00:01   1.56 MB/s
Extracting packages ...
[      COMPLETE      ]|###################################################| 100%
Linking packages ...
[      COMPLETE      ]|###################################################| 100%


In [12]:
!pip install jupyter-sql

In [1]:
%load_ext sql

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [10]:
%debug

> /Users/decibel/miniconda3/envs/PGDay/lib/python3.5/site-packages/IPython/core/extensions.py(88)load_extension()
     86                 self.loaded.add(module_str)
     87             else:
---> 88                 return "no load function"
     89 
     90     def unload_extension(self, module_str):

ipdb> p self.ipython_extension_dir
'/Users/decibel/.ipython/extensions'
ipdb> h

Documented commands (type help <topic>):
EOF    c          d        h         next    pp       retval  u          whatis
a      cl         debug    help      p       psource  run     unalias    where 
alias  clear      disable  ignore    pdef    q        rv      undisplay
args   commands   display  interact  pdoc    quit     s       unt      
b      condition  down     j         pfile   r        source  until    
break  cont       enable   jump      pinfo   restart  step    up       
bt     continue   exit     n         pinfo2  return   tbreak  w        

Miscellaneous help topics:
exec  pdb

Undocumented co

In [8]:
%magic